# Implement a function to query json_schema

The goal of this excercise is to implement a function that allows you to query the type of a keypath in a jsonschema. This function will accept a valid jsonschema (https://json-schema.org/specification.html) as dict, a key_path (eg: `foo.bar.baz`) and return the `type` of the property. 

Note:

1. There are only two fields in the schema you have to pay attention to: `properties` and `definitions`.
2. if the dictionary associated with the field has a field named `$ref` it means that its refering to another schema stored under the top level schema. You have follow the link to get to the actual definition. 
3. For the sake of this excercise you can assume that all values for `$ref` will start wiht `#/<key_path>`.
4. You should see a schema and some assert statements under "Test" section. You should NOT have to change anything under the "Test" section. If you can get our code to pass the tests, then it means your function works as expected.

Feel free to use whatever libraries you need to use except any libraries that actually allow you to query the jsonschema

In [1]:
def get_type_for_key_path(schema: dict, key_path: str) -> str: 
    ''' 
    Searches the schema for the data type for a given path
    
    Args:
        schema: dict  - A dictionary representing the json doc
        key_path: str - The key search path
        
    Returns:
       str - The type of the key or None if key was not found
    '''
    
    # Get the type for each field in the schema
    path_type_dict = parse_schema_types(schema, ref_path=None, name=None)
    
    # return the data type for the key_path or None
    return path_type_dict.get(key_path,None)

    
def parse_schema_types(schema: dict, ref_path: str, name: str) -> dict:
    ''' 
    Parses and builds a dictionary containing the 
    
    Args:
        schema: dict  - A dictionary representing the json doc
        ref_path: str - The parsed key_path
        name: str     - The alias field name
        
    Returns:
       dict - Dictionay containing the data types for each field in the schema
    '''
    
    type_dict = {}
    ref = "$ref"
    
    
    sub_schema = schema 
    
    # if the ref_path is defined, then need to obtain definition for the reference
    if ref_path is not None:
        sub_schema = schema["definitions"][ref_path.split('/')[-1]]
    
    # for each field in the schema
    for k,v in sub_schema['properties'].items():
        
        # assign the key_name to the key
        key_name = k
        
        # if the field has an alias, replace th
        if name is not None:
            key_name = ".".join((name,k))
            
        # if the value is a dictionan and contains a References
        # then 
        #    update the dictionary with the results from the recursive call
        #    to parse_schema_type to obtain the sub schema 
        # else
        #    return the type from the given key
        if isinstance(v,dict) and ref in v:
            # update the dictionay with the results from the recursive call
            type_dict.update(parse_schema_types(schema,v[ref],key_name))
        else:
            type_dict[key_name] = v["type"] 
    
    # Return the type dictionary containing the datatypes for all the fields in the schema
    return type_dict


# Test

In [2]:
import json

schema = json.loads('''{
  "$id": "https://example.com/nested-schema.json",
  "title": "nested-schema",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "required": [
    "EmploymentInformation",
    "EmployeePartyID",
    "Age"
  ],
  "properties": {
    "EmployeePartyID": {
      "type": "string",
      "minLength": 1,
      "maxLength": 3
    },
    "EmploymentInformation": {
      "$ref": "#/definitions/EmploymentInformation"
    },
    "Age": {
      "type": "integer",
      "minimum": 16,
      "maximum": 80
    }
  },
  "definitions": {
    "EmploymentInformation": {
      "type": "object",
      "required": [
        "OriginalHireDate"
      ],
      "properties": {
        "OriginalHireDate": {
          "type": "string",
          "format": "date"
        },
        "Beneficiary": {
          "$ref": "#/definitions/DependantInformation"
        }
      }
    },
    "DependantInformation": {
      "type": "object",
      "required": [
        "Name"
      ],
      "properties": {
        "Name": {
          "type": "string",
          "minLength": 5
        }
      }
    }
  },
  "description": "nested-schema"
}''')

assert(get_type_for_key_path(schema, "Age") == "integer")
assert(get_type_for_key_path(schema, "EmploymentInformation.OriginalHireDate") == "string")
assert(get_type_for_key_path(schema, "EmploymentInformation.Beneficiary.Name") == "string")
assert(get_type_for_key_path(schema, "foo.bar") == None)